In [6]:
%session_id_prefix iceberg-sql-
%glue_version 3.0
%idle_timeout 60
%connections demo_iceberg_connector
%%configure 
{
  "--conf": "spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions"
}

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 0.37.0 
Setting session ID prefix to iceberg-sql-
Setting Glue version to: 3.0
Current idle_timeout is 2880 minutes.
idle_timeout has been set to 60 minutes.
Connections to be included:
demo_iceberg_connector
The following configurations have been updated: {'--conf': 'spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions'}


In [1]:
catalog_name = "glue_catalog"
bucket_name = "demo-dbt-output-788996449432"
bucket_prefix = "jaffle_db"
database_name = "jaffle_db"
warehouse_path = f"s3://{bucket_name}/{bucket_prefix}"
dynamodb_table = 'myGlueLockTable'

Authenticating with environment variables and user-defined glue_role_arn: arn:aws:iam::788996449432:role/AWSGlueServiceRole-dbt
Trying to create a Glue session for the kernel.
Worker Type: G.1X
Number of Workers: 5
Session ID: dab82f08-455d-4e6c-8812-bbc224d2eabc
Job Type: glueetl
Applying the following default arguments:
--glue_kernel_version 0.37.0
--enable-glue-datacatalog true
--conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions
Waiting for session dab82f08-455d-4e6c-8812-bbc224d2eabc to get into ready status...
Session dab82f08-455d-4e6c-8812-bbc224d2eabc has been created.



## Initialize SparkSession

In [13]:
spark.stop()
sc.stop()

In [14]:
from pyspark.sql import SparkSession
spark = SparkSession.builder \
    .config("spark.sql.warehouse.dir", warehouse_path) \
    .config(f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog") \
    .config(f"spark.sql.catalog.{catalog_name}.warehouse", warehouse_path) \
    .config(f"spark.sql.catalog.{catalog_name}.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog") \
    .config(f"spark.sql.catalog.{catalog_name}.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config(f"spark.sql.catalog.{catalog_name}.lock-impl", "org.apache.iceberg.aws.glue.DynamoLockManager") \
    .config(f"spark.sql.catalog.{catalog_name}.lock.table", dynamodb_table) \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .getOrCreate()

In [6]:
%%sql 
USE jaffle_db

++
||
++
++


In [7]:
%%sql
SHOW TABLES

+---------+-------------+-----------+
| database|    tableName|isTemporary|
+---------+-------------+-----------+
|jaffle_db|raw_customers|      false|
|jaffle_db|   raw_orders|      false|
|jaffle_db| raw_payments|      false|
|jaffle_db|stg_customers|      false|
|jaffle_db|   stg_orders|      false|
|jaffle_db| stg_payments|      false|
|jaffle_db|    customers|      false|
|jaffle_db|       orders|      false|
+---------+-------------+-----------+


## Run before and after upsert: Insert Mehdi M. with customer id 101 and some orders, update last name of Christina from R. to P.

In [15]:
%%sql
select customer_id \
,first_name \
,last_name \
,first_order \
,most_recent_order \
,number_of_orders \
,customer_lifetime_value \
from glue_catalog.jaffle_db.customers \
where customer_id in (84,101)

+-----------+----------+---------+-----------+-----------------+----------------+-----------------------+
|customer_id|first_name|last_name|first_order|most_recent_order|number_of_orders|customer_lifetime_value|
+-----------+----------+---------+-----------+-----------------+----------------+-----------------------+
|      101.0|     Mehdi|       M.| 2022-10-04|       2022-15-05|               3|                   17.0|
|       84.0| Christina|       P.| 2018-01-17|       2018-04-02|               2|                   36.0|
+-----------+----------+---------+-----------+-----------------+----------------+-----------------------+


## View History and Snapshots

In [16]:
%%sql
SELECT * FROM glue_catalog.jaffle_db.customers.history

+--------------------+-------------------+-------------------+-------------------+
|     made_current_at|        snapshot_id|          parent_id|is_current_ancestor|
+--------------------+-------------------+-------------------+-------------------+
|2023-01-05 16:45:...|5979839008058067040|               null|               true|
|2023-01-05 16:57:...|5450052278098493700|5979839008058067040|               true|
+--------------------+-------------------+-------------------+-------------------+


In [17]:
%%sql
SELECT * FROM glue_catalog.jaffle_db.customers.snapshots

+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|        committed_at|        snapshot_id|          parent_id|operation|       manifest_list|             summary|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+
|2023-01-05 16:45:...|5979839008058067040|               null|   append|s3://demo-dbt-out...|{spark.app.id -> ...|
|2023-01-05 16:57:...|5450052278098493700|5979839008058067040|overwrite|s3://demo-dbt-out...|{spark.app.id -> ...|
+--------------------+-------------------+-------------------+---------+--------------------+--------------------+


In [18]:
%%sql
SELECT h.made_current_at, s.operation, h.snapshot_id, h.is_current_ancestor, s.summary["spark.app.id"] FROM glue_catalog.jaffle_db.customers.history h JOIN glue_catalog.jaffle_db.customers.snapshots s  ON h.snapshot_id = s.snapshot_id ORDER BY made_current_at

+--------------------+---------+-------------------+-------------------+---------------------+
|     made_current_at|operation|        snapshot_id|is_current_ancestor|summary[spark.app.id]|
+--------------------+---------+-------------------+-------------------+---------------------+
|2023-01-05 16:45:...|   append|5979839008058067040|               true| spark-application...|
|2023-01-05 16:57:...|overwrite|5450052278098493700|               true| spark-application...|
+--------------------+---------+-------------------+-------------------+---------------------+


## Stop Session

In [ ]:
%stop_session